# 4) iPyAladin
``pip``
``jupyter nbextension enable --py --sys-prefix ipyaladin``


In [8]:
import ipyaladin as ipyal

In [9]:
aladin = ipyal.Aladin()
aladin

A Jupyter Widget

In [10]:
aladin.target= 'M100'

In [11]:
aladin.survey= 'P/allWISE/color'

In [12]:
aladin.set_color_map('rainbow')

In [13]:
url= 'http://vizier.u-strasbg.fr/viz-bin/votable?-source=HIP2&-c=LMC&-out.add=_RAJ,_DEJ&-oc.form=dm&-out.meta=DhuL&-out.max=9999&-c.rm=180'
options= {'sourceSize':12, 'color': '#f08080'}
aladin.add_catalog_from_URL(url, options)

In [14]:
aladin.add_moc_from_URL('http://alasky.u-strasbg.fr/footprints/tables/vizier/II_337_vvv1/MOC?nside=256', {'color': 'violet', 'opacity': 0.3})


In [15]:
aladin.fov

59.91528

In [16]:
from ipyaladin import Aladin

from ipywidgets import Layout, Box, widgets

a = Aladin(layout=Layout(width='33.33%'), target='M 81', fov=0.3)
b = Aladin(layout=Layout(width='33.33%'), survey='P/DSS2/red')
c = Aladin(layout=Layout(width='33.33%'), survey='P/2MASS/color')

# synchronize target between 3 widgets
widgets.jslink((a, 'target'), (b, 'target'))
widgets.jslink((b, 'target'), (c, 'target'))

# synchronize FoV (zoom level) between 3 widgets
widgets.jslink((a, 'fov'), (b, 'fov'))
widgets.jslink((b, 'fov'), (c, 'fov'))

items = [a, b, c]

box_layout = Layout(display='flex',
                    flex_flow='row',
                    align_items='stretch',
                    border='solid',
                    width='100%')
box = Box(children=items, layout=box_layout)
box

A Jupyter Widget

In [17]:
from ipyaladin import Aladin

from ipywidgets import Layout, Box, widgets

aladin = Aladin(layout=Layout(width='70%'), target='M 36', fov=0.3)
info = widgets.HTML()


box_layout = Layout(display='flex',
                    flex_flow='row',
                    align_items='stretch',
                    width='100%')
box = Box(children=[aladin, info], layout=box_layout)
box

A Jupyter Widget

In [18]:
import requests
def process_result(data):
    info.value = ''
    ra = data['ra']
    dec = data['dec']
    radius = min(aladin.fov / 10, 5)
    query = """SELECT TOP 1 main_id, ra, dec, DISTANCE(POINT('ICRS', %f, %f), POINT('ICRS', ra, dec)) as d FROM basic
               WHERE CONTAINS(POINT('ICRS', ra, dec), CIRCLE('ICRS', %f, %f, %f))=1
               ORDER BY d ASC""" % (ra, dec, ra, dec, aladin.fov / 10)
    
    r = requests.get('http://simbad.u-strasbg.fr/simbad/sim-tap/sync', params={'query': query, 'request': 'doQuery', 'lang': 'adql', 'format': 'json', 'phase': 'run'})
    obj_name = ''
    obj_coo = None
    obj_data = r.json()['data']
    if len(obj_data)==0:
        return
    
    obj_name = obj_data[0][0]
    obj_coo = [obj_data[0][1], obj_data[0][2]]
        
    sed_img = '<img src="http://cdsportal.u-strasbg.fr/cgi-bin/PhotVizPreview/plot?ra=%f&dec=%f&radius_arcsec=5&w=200&h=150&point_size=4">' % (obj_coo[0], obj_coo[1])
    info.value =  '<h2>%s</h2><br><br>%s' % (obj_name, sed_img)
    
aladin.add_listener('click', process_result)
